In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

# Perform a SQL query to find the average price for a four-bedroom house sold per year
query = """
SELECT year(date_sold) AS year, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year(date_sold)
ORDER BY year(date_sold)
"""

# Run the query and convert to a Pandas DataFrame for better visualization
avg_price_per_year_df = spark.sql(query).toPandas()

# Show the results
avg_price_per_year_df


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# Perform a SQL query to find the average price for a home with three bedrooms and three bathrooms per year built
query = """
SELECT year_built, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
"""

# Run the query and show results
avg_price_per_year_built_df = spark.sql(query).toPandas()

# Display the results
avg_price_per_year_built_df


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# Perform a SQL query to find the average price for a home with 3 bedrooms, 3 bathrooms, two floors, 
# and greater than or equal to 2,000 square feet per year built
query = """
SELECT year_built, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 
      AND bathrooms = 3 
      AND floors = 2 
      AND sqft >= 2000
GROUP BY year_built
ORDER BY year_built
"""

# Run the query and show results
avg_price_criteria_df = spark.sql(query).toPandas()

# Display the results
avg_price_criteria_df


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

# Start time
start_time = time.time()

# Perform a SQL query to find the average price per "view" rating for homes with an average price >= $350,000
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Run the query
avg_price_view_df = spark.sql(query)

# Action to trigger the query execution and measure the run time
avg_price_view_df.show()

# End time and runtime calculation
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

# To check if the table is cached, you can list all cached tables.
# # This will return a list of cached tables.
# cached_tables = spark.catalog.isCached("home_sales")
# print(f"Is the 'home_sales' table cached? {cached_tables}")


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.
# Start timing
start_time = time.time()

# SQL query using the cached 'home_sales' table
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Run the query using the cached data
avg_price_view_cached_df = spark.sql(query)

# Trigger the action to execute the query and measure the run time
avg_price_view_cached_df.show()

# Calculate the time it took to run the query
runtime = time.time() - start_time
print("--- %s seconds ---" % runtime)



In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

# Assuming 'home_sales' DataFrame is already defined and available

# Write out the data partitioned by 'date_built' field in Parquet format
home_sales.write.partitionBy("date_built").parquet("home_sales_parquet")

# Note: Replace "path/to/save/home_sales_parquet" with the actual path where you want to save the Parquet files.


In [ ]:
# 11. Read the formatted parquet data.

# Read the partitioned Parquet data
parquet_home_sales = spark.read.parquet("home_sales_parquet")

# Show some data to verify it's been read correctly
parquet_home_sales.show()

# Note: Replace "path/to/save/home_sales_parquet" with the actual path where your Parquet files are saved.


In [ ]:
# 12. Create a temporary table for the parquet data.

# Assuming 'parquet_home_sales' is your DataFrame from the Parquet data

# Create a temporary view of the DataFrame
parquet_home_sales.createOrReplaceTempView("parquet_home_sales_view")

# Now you can use SQL queries with SparkSQL on this view
# For example, to show the first few rows:
spark.sql("SELECT * FROM parquet_home_sales_view LIMIT 5").show()


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

# Start timing
start_time = time.time()

# SQL query using the temporary table created from the Parquet DataFrame
query = """
SELECT view, 
       ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales_view
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Run the query
avg_price_view_parquet_df = spark.sql(query)

# Trigger the action to execute the query and measure the run time
avg_price_view_parquet_df.show()

# Calculate the time it took to run the query
runtime = time.time() - start_time
print("--- %s seconds ---" % runtime)

In [ ]:
# 14. Uncache the home_sales temporary table.


# Uncache the 'home_sales' temporary table
spark.catalog.uncacheTable("home_sales")

# Verify if the table is uncached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is the 'home_sales' table cached? {is_cached}")


In [ ]:
# 15. Check if the home_sales is no longer cached

# Check if the 'home_sales' table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is the 'home_sales' table cached? {is_cached}")
